<a href="https://colab.research.google.com/github/sunohk/dataon_pj/blob/main/3_news_kpfbert_summary(%EB%89%B4%EC%8A%A4%EC%9A%94%EC%95%BD).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kpf-Bert를 이용한 뉴스기사 3줄요약 서비스  

변곡점 기반 +- 1 day의 댓글 수가 많은 기사 3개씩 추출 및 요약

목표: 여론이 뒤바뀌거나 언급량이 급증한 시기의 뉴스 요약본을 확인하여 이유를 추측
정책과 연관있는 기사의 요약본을 통해 국회는 여론 파악



In [ ]:
from google.colab import drive
drive.mount('./gdrive')

Mounted at ./gdrive


## Set up

In [ ]:
#@title Library

!pip install git+https://github.com/PyTorchLightning/pytorch-lightning
!pip install git+https://github.com/KPFBERT/kpfbert
!pip install transformers
!pip install --upgrade pytorch-lightning
!pip install torchmetrics
!pip install kss

  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-cez40urf
  Running command git clone --filter=blob:none --quiet https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-cez40urf
  Resolved https://github.com/PyTorchLightning/pytorch-lightning to commit e7afe04ee86b64c76a5446088b3b75d9c275e5bf
  Running command git submodule update --init --recursive -q
  Encountered 22 file(s) that should have been pointers, but weren't:
        .notebooks/course_UvA-DL/01-introduction-to-pytorch.ipynb
        .notebooks/course_UvA-DL/02-activation-functions.ipynb
        .notebooks/course_UvA-DL/03-initialization-and-optimization.ipynb
        .notebooks/course_UvA-DL/04-inception-resnet-densenet.ipynb
        .notebooks/course_UvA-DL/05-transformers-and-MH-attention.ipynb
        .notebooks/course_UvA-DL/06-graph-neural-networks.ipynb
        .notebooks/course_UvA-DL/07-deep-energy-based-generative-models.ipynb
        .notebooks/course_UvA-DL/08-

In [ ]:
import math
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim

from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

from torch.nn.init import xavier_uniform_

import pytorch_lightning as pl
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import kss

%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

pl.seed_everything(RANDOM_SEED)

INFO:lightning_fabric.utilities.seed:Seed set to 42


42

In [ ]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.8.6
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.4.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.1.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.13.0
backcall                         0.2.0
beautifulsoup4                   4.11.2
bidict                           0.22.1
b

In [ ]:
MAX_TOKEN_COUNT = 512
N_EPOCHS = 5
BATCH_SIZE = 4

In [ ]:
#@title tokenizer
# kpfBERT 토크나이저는 형태소와 유사하게 잘 토크나이징을 하게 설계되어 있다.
from transformers import BertTokenizerFast, BertModel

BERT_MODEL_NAME = "jinmang2/kpfbert"
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

In [ ]:
class SummDataset(Dataset):

    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: BertTokenizer,
        max_token_len: int = 512
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        tokenlist = []
        for sent in data_row.article_original:
            tokenlist.append(tokenizer(
                text = sent,
                add_special_tokens = True)) #, # Add '[CLS]' and '[SEP]'

        src = [] # 토크나이징 된 전체 문단
        labels = []  # 요약문에 해당하면 1, 아니면 0으로 문장수 만큼 생성
        segs = []  #각 토큰에 대해 홀수번째 문장이면 0, 짝수번째 문장이면 1을 매핑
        clss = []  #[CLS]토큰의 포지션값을 지정

        odd = 0
        for tkns in tokenlist:
            if odd > 1 : odd = 0
            clss = clss + [len(src)]
            src = src + tkns['input_ids']
            segs = segs + [odd] * len(tkns['input_ids'])
            if tokenlist.index(tkns) in data_row.extractive :
                labels = labels + [1]
            else:
                labels = labels + [0]
            odd += 1

            #truncation
            if len(src) == MAX_TOKEN_COUNT:
                break
            elif len(src) > MAX_TOKEN_COUNT:
                src = src[:self.max_token_len - 1] + [src[-1]]
                segs = segs[:self.max_token_len]
                break

        #padding
        if len(src) < MAX_TOKEN_COUNT:
            src = src + [0]*(self.max_token_len - len(src))
            segs = segs + [0]*(self.max_token_len - len(segs))

        if len(clss) < MAX_TOKEN_COUNT:
            clss = clss + [-1]*(self.max_token_len - len(clss))
        if len(labels) < MAX_TOKEN_COUNT:
            labels = labels + [0]*(self.max_token_len - len(labels))

        return dict(
            src = torch.tensor(src),
            segs = torch.tensor(segs),
            clss = torch.tensor(clss),
            labels= torch.FloatTensor(labels)
        )

In [ ]:
class SummDataModule(pl.LightningDataModule):

    def __init__(self, train_df, test_df, val_df, tokenizer, batch_size=1, max_token_len=512):
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.test_df = test_df
        self.val_df = val_df
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def setup(self, stage=None):
        self.train_dataset = SummDataset(
            self.train_df,
            self.tokenizer,
            self.max_token_len
        )

        self.test_dataset = SummDataset(
            self.test_df,
            self.tokenizer,
            self.max_token_len
        )

        self.val_dataset = SummDataset(
            self.val_df,
            self.tokenizer,
            self.max_token_len
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=8 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            num_workers=8 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

    def test_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=8 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, dropout, dim, max_len=5000):
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp((torch.arange(0, dim, 2, dtype=torch.float) *
                              -(math.log(10000.0) / dim)))
        pe[:, 0::2] = torch.sin(position.float() * div_term)
        pe[:, 1::2] = torch.cos(position.float() * div_term)
        pe = pe.unsqueeze(0)
        super(PositionalEncoding, self).__init__()
        self.register_buffer('pe', pe)
        self.dropout = nn.Dropout(p=dropout)
        self.dim = dim

    def forward(self, emb, step=None):
        emb = emb * math.sqrt(self.dim)
        if (step):
            emb = emb + self.pe[:, step][:, None, :]

        else:
            emb = emb + self.pe[:, :emb.size(1)]
        emb = self.dropout(emb)
        return emb

    def get_emb(self, emb):
        return self.pe[:, :emb.size(1)]

In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, heads, d_ff, dropout):
        super(TransformerEncoderLayer, self).__init__()

        self.self_attn = MultiHeadedAttention(
            heads, d_model, dropout=dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, iter, query, inputs, mask):
        if (iter != 0):
            input_norm = self.layer_norm(inputs)
        else:
            input_norm = inputs

        mask = mask.unsqueeze(1)
        context = self.self_attn(input_norm, input_norm, input_norm,
                                 mask=mask)
        out = self.dropout(context) + inputs
        return self.feed_forward(out)

In [ ]:
class ExtTransformerEncoder(nn.Module):
    def __init__(self, hidden_size=768, d_ff=2048, heads=8, dropout=0.2, num_inter_layers=2):
        super(ExtTransformerEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_inter_layers = num_inter_layers
        self.pos_emb = PositionalEncoding(dropout, hidden_size)
        self.transformer_inter = nn.ModuleList(
            [TransformerEncoderLayer(hidden_size, heads, d_ff, dropout)
            for _ in range(num_inter_layers)])
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_size, eps=1e-6)
        self.wo = nn.Linear(hidden_size, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, top_vecs, mask):
        """ See :obj:`EncoderBase.forward()`"""

        batch_size, n_sents = top_vecs.size(0), top_vecs.size(1)
        pos_emb = self.pos_emb.pe[:, :n_sents]
        x = top_vecs * mask[:, :, None].float()
        x = x + pos_emb

        for i in range(self.num_inter_layers):
            x = self.transformer_inter[i](i, x, x, ~mask)

        x = self.layer_norm(x)
        sent_scores = self.sigmoid(self.wo(x))
        sent_scores = sent_scores.squeeze(-1) * mask.float()

        return sent_scores

In [ ]:
class PositionwiseFeedForward(nn.Module):
    """ A two-layer Feed-Forward-Network with residual layer norm.

    Args:
        d_model (int): the size of input for the first-layer of the FFN.
        d_ff (int): the hidden layer size of the second-layer
            of the FNN.
        dropout (float): dropout probability in :math:`[0, 1)`.
    """

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def gelu(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))


    def forward(self, x):
        inter = self.dropout_1(self.gelu(self.w_1(self.layer_norm(x))))
        output = self.dropout_2(self.w_2(inter))
        return output + x

In [ ]:
class MultiHeadedAttention(nn.Module):
    """
    Multi-Head Attention module from
    "Attention is All You Need"
    :cite:`DBLP:journals/corr/VaswaniSPUJGKP17`.

    Similar to standard `dot` attention but uses
    multiple attention distributions simulataneously
    to select relevant items.

    .. mermaid::

       graph BT
          A[key]
          B[value]
          C[query]
          O[output]
          subgraph Attn
            D[Attn 1]
            E[Attn 2]
            F[Attn N]
          end
          A --> D
          C --> D
          A --> E
          C --> E
          A --> F
          C --> F
          D --> O
          E --> O
          F --> O
          B --> O

    Also includes several additional tricks.

    Args:
       head_count (int): number of parallel heads
       model_dim (int): the dimension of keys/values/queries,
           must be divisible by head_count
       dropout (float): dropout parameter
    """

    def __init__(self, head_count, model_dim, dropout=0.1, use_final_linear=True):
        assert model_dim % head_count == 0
        self.dim_per_head = model_dim // head_count
        self.model_dim = model_dim

        super(MultiHeadedAttention, self).__init__()
        self.head_count = head_count

        self.linear_keys = nn.Linear(model_dim,
                                     head_count * self.dim_per_head)
        self.linear_values = nn.Linear(model_dim,
                                       head_count * self.dim_per_head)
        self.linear_query = nn.Linear(model_dim,
                                      head_count * self.dim_per_head)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        self.use_final_linear = use_final_linear
        if (self.use_final_linear):
            self.final_linear = nn.Linear(model_dim, model_dim)

    def forward(self, key, value, query, mask=None,
                layer_cache=None, type=None, predefined_graph_1=None):
        """
        Compute the context vector and the attention vectors.

        Args:
           key (`FloatTensor`): set of `key_len`
                key vectors `[batch, key_len, dim]`
           value (`FloatTensor`): set of `key_len`
                value vectors `[batch, key_len, dim]`
           query (`FloatTensor`): set of `query_len`
                 query vectors  `[batch, query_len, dim]`
           mask: binary mask indicating which keys have
                 non-zero attention `[batch, query_len, key_len]`
        Returns:
           (`FloatTensor`, `FloatTensor`) :

           * output context vectors `[batch, query_len, dim]`
           * one of the attention vectors `[batch, query_len, key_len]`
        """

        batch_size = key.size(0)
        dim_per_head = self.dim_per_head
        head_count = self.head_count
        key_len = key.size(1)
        query_len = query.size(1)

        def shape(x):
            """  projection """
            return x.view(batch_size, -1, head_count, dim_per_head) \
                .transpose(1, 2)

        def unshape(x):
            """  compute context """
            return x.transpose(1, 2).contiguous() \
                .view(batch_size, -1, head_count * dim_per_head)

        # 1) Project key, value, and query.
        if layer_cache is not None:
            if type == "self":
                query, key, value = self.linear_query(query), \
                                    self.linear_keys(query), \
                                    self.linear_values(query)

                key = shape(key)
                value = shape(value)

                if layer_cache is not None:
                    device = key.device
                    if layer_cache["self_keys"] is not None:
                        key = torch.cat(
                            (layer_cache["self_keys"].to(device), key),
                            dim=2)
                    if layer_cache["self_values"] is not None:
                        value = torch.cat(
                            (layer_cache["self_values"].to(device), value),
                            dim=2)
                    layer_cache["self_keys"] = key
                    layer_cache["self_values"] = value
            elif type == "context":
                query = self.linear_query(query)
                if layer_cache is not None:
                    if layer_cache["memory_keys"] is None:
                        key, value = self.linear_keys(key), \
                                     self.linear_values(value)
                        key = shape(key)
                        value = shape(value)
                    else:
                        key, value = layer_cache["memory_keys"], \
                                     layer_cache["memory_values"]
                    layer_cache["memory_keys"] = key
                    layer_cache["memory_values"] = value
                else:
                    key, value = self.linear_keys(key), \
                                 self.linear_values(value)
                    key = shape(key)
                    value = shape(value)
        else:
            key = self.linear_keys(key)
            value = self.linear_values(value)
            query = self.linear_query(query)
            key = shape(key)
            value = shape(value)

        query = shape(query)

        key_len = key.size(2)
        query_len = query.size(2)

        # 2) Calculate and scale scores.
        query = query / math.sqrt(dim_per_head)
        scores = torch.matmul(query, key.transpose(2, 3))

        if mask is not None:
            mask = mask.unsqueeze(1).expand_as(scores)
            scores = scores.masked_fill(mask, -1e18) # how can i fix it to use fp16...

        # 3) Apply attention dropout and compute context vectors.

        attn = self.softmax(scores)

        if (not predefined_graph_1 is None):
            attn_masked = attn[:, -1] * predefined_graph_1
            attn_masked = attn_masked / (torch.sum(attn_masked, 2).unsqueeze(2) + 1e-9)

            attn = torch.cat([attn[:, :-1], attn_masked.unsqueeze(1)], 1)

        drop_attn = self.dropout(attn)
        if (self.use_final_linear):
            context = unshape(torch.matmul(drop_attn, value))
            output = self.final_linear(context)
            return output
        else:
            context = torch.matmul(drop_attn, value)
            return context


In [ ]:
class Summarizer(pl.LightningModule):

    def __init__(self, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.max_pos = 512
        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME) #, return_dict=True)
        self.ext_layer = ExtTransformerEncoder()
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.loss = nn.BCELoss(reduction='none')

        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.test_step_outputs = []

        for p in self.ext_layer.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

    def forward(self, src, segs, clss, labels=None): #, input_ids, attention_mask, labels=None):

        mask_src = ~(src == 0) #1 - (src == 0)
        mask_cls = ~(clss == -1) #1 - (clss == -1)

        top_vec = self.bert(src, token_type_ids=segs, attention_mask=mask_src)
        top_vec = top_vec.last_hidden_state

        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
        sents_vec = sents_vec * mask_cls[:, :, None].float()

        sent_scores = self.ext_layer(sents_vec, mask_cls).squeeze(-1)

        loss = 0
        if labels is not None:
            loss = self.loss(sent_scores, labels)

            loss = (loss * mask_cls.float()).sum() / len(labels)

        return loss, sent_scores

    def step(self, batch):

        src = batch['src']
        if len(batch['labels']) > 0 :
            labels = batch['labels']
        else:
            labels = None
        segs = batch['segs']
        clss = batch['clss']

        loss, sent_scores = self(src, segs, clss, labels)

        return loss, sent_scores, labels

    def training_step(self, batch, batch_idx):

        loss, sent_scores, labels = self.step(batch)
        self.training_step_outputs.append(loss)
        self.log("train_loss", loss, prog_bar=True, logger=True)

        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def validation_step(self, batch, batch_idx):

        loss, sent_scores, labels = self.step(batch)
        self.validation_step_outputs.append(loss)
        self.log("val_loss", loss, prog_bar=True, logger=True)

        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def test_step(self, batch, batch_idx):

        loss, sent_scores, labels = self.step(batch)
        self.test_step_outputs.append(loss)
        self.log("test_loss", loss, prog_bar=True, logger=True)

        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def on_train_epoch_end(self):

        epoch_average = torch.stack(self.training_step_outputs).mean()
        self.log("training_epoch_average", epoch_average)
        self.training_step_outputs.clear()  # free memory

    def on_validation_epoch_end(self):
        epoch_average = torch.stack(self.validation_step_outputs).mean()
        self.log("validation_epoch_average", epoch_average)
        self.validation_step_outputs.clear()  # free memory

    def on_test_epoch_end(self):
        epoch_average = torch.stack(self.test_step_outputs).mean()
        self.log("test_epoch_average", epoch_average)
        self.test_step_outputs.clear()  # free memory

    def configure_optimizers(self):

        optimizer = AdamW(self.parameters(), lr=2e-5)

        steps_per_epoch=len(train_df) // BATCH_SIZE
        total_training_steps = steps_per_epoch * N_EPOCHS

        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=steps_per_epoch,
            num_training_steps=total_training_steps
        )

        return dict(
            optimizer=optimizer,
            lr_scheduler=dict(
                scheduler=scheduler,
                interval='step'
            )
        )

In [ ]:
model = Summarizer()

Some weights of BertModel were not initialized from the model checkpoint at jinmang2/kpfbert and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Model Load

In [ ]:
PATH = './gdrive/MyDrive/01_공모전/2023_News/5_Presentation/제출자료/모델/2_kpfbertsumm_trained_model.pt'

device = torch.device("cuda")
trained_model = Summarizer()
trained_model.load_state_dict(torch.load(PATH, map_location="cuda:0"))  # 사용할 GPU 장치 번호를 선택합니다.
trained_model.to(device)

Some weights of BertModel were not initialized from the model checkpoint at jinmang2/kpfbert and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summarizer(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(36440, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [ ]:
#@title 뉴스데이터 preprocess

def data_process(text):
    # 문장 분리 하고,
    sents = kss.split_sentences(text)

    #데이터 가공하고,
    tokenlist = []
    for sent in sents:
        tokenlist.append(tokenizer(
            text = sent,
            add_special_tokens = True)) #, # Add '[CLS]' and '[SEP]'

    src = [] # 토크나이징 된 전체 문단
    labels = []  # 요약문에 해당하면 1, 아니면 0으로 문장수 만큼 생성
    segs = []  #각 토큰에 대해 홀수번째 문장이면 0, 짝수번째 문장이면 1을 매핑
    clss = []  #[CLS]토큰의 포지션값을 지정

    odd = 0

    for tkns in tokenlist:

        if odd > 1 : odd = 0
        clss = clss + [len(src)]
        src = src + tkns['input_ids']
        segs = segs + [odd] * len(tkns['input_ids'])
        odd += 1

        #truncation
        if len(src) == MAX_TOKEN_COUNT:
            break
        elif len(src) > MAX_TOKEN_COUNT:
            src = src[:MAX_TOKEN_COUNT - 1] + [src[-1]]
            segs = segs[:MAX_TOKEN_COUNT]
            break

    #padding
    if len(src) < MAX_TOKEN_COUNT:
        src = src + [0]*(MAX_TOKEN_COUNT - len(src))
        segs = segs + [0]*(MAX_TOKEN_COUNT - len(segs))

    if len(clss) < MAX_TOKEN_COUNT:
        clss = clss + [-1]*(MAX_TOKEN_COUNT - len(clss))

    return dict(
        sents = sents, #정답 출력을 위해...
        src = torch.tensor(src).cuda(),
        segs = torch.tensor(segs).cuda(),
        clss = torch.tensor(clss).cuda(),
    )

In [ ]:
def summarize_test(text):
    data = data_process(text.replace('\n',''))

    #trained_model에 넣어 결과값 반환
    _, rtn = trained_model(data['src'].unsqueeze(0), data['segs'].unsqueeze(0), data['clss'].unsqueeze(0))
    rtn = rtn.squeeze()

    # 예측 결과값을 받기 위한 프로세스
    rtn_sort, idx = rtn.sort(descending = True)

    rtn_sort = rtn_sort.tolist()
    idx = idx.tolist()

    end_idx = rtn_sort.index(0)

    rtn_sort = rtn_sort[:end_idx]
    idx = idx[:end_idx]

    if len(idx) > 3:
        rslt = idx[:3]
    else:
        rslt = idx

    summ = []
    # print(' *** 입력한 문단의 요약문은 ...')
    for i, r in enumerate(rslt):
        summ.append(data['sents'][r])
        print('[', i+1, ']', summ[i])

    return summ

## Load news data

In [ ]:
raw_df = pd.read_excel('./gdrive/MyDrive/01_공모전/2023_News/5_Presentation/제출자료/데이터/Processed_Data/3_news_summ_pre.xlsx')
df = raw_df

In [ ]:
#processing
#combine title + content
# df['summary'] = df['제목'] + df['내용'].astype(str)
# keep_cols = ['작성일','작성자','제목','내용']
# df = raw_df[keep_cols]
# df.columns = ['Date', 'Publisher', 'Title', 'Text']
df['Text'] = df['Text'].str.replace('…', '').str.replace('\\', '')

<ipython-input-25-c8d47bf94919>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Text'] = df['Text'].str.replace('…', '').str.replace('\\', '')


In [ ]:
test1= df['Text'][0]
test1


'"가르치려 한 오만함, 청년과 단절 원인" 반성 "박원순 피해자 제대로 된 사과와 반성 이뤄지지 않았어" 이소영 의원을 비롯한 더불어민주당 2030의원들이 9일 오전 서울 여의도 국회 소통관에서 \'더불어민주당 2030의원 입장문\' 발표를 기자회견을 하고 있다. 2021.4.9/뉴스1 © News1 박세연 기자 (서울=뉴스1) 서혜림 기자 = 더불어민주당 2030대 의원들은 9일 입장문을 내고 "돌아선 국민의 마음의 원인은 저희를 포함한 민주당의 착각과 오판에 있었음을 자인한다"고 말했다. 민주당 오영환·이소영·장경태·장철민·전용기 의원은 이날 오전 국회 소통관에서 기자회견을 열고 "선거 유세 현장과 삶의 현장에서 만난 20대 30대 청년들은 민주당에 싸늘하고 무관심했고 지난 1년 동안 많은 분의 마음이 돌아섰음을 현장에서 느꼈다"고 말했다. 이들은 "이번 재보궐선거를 치르게 된 원인이 우리 당 공직자의 성 비위 문제였음에도 불구하고 우리 당은 당헌·당규를 개정해 후보를 내고 피해자에 대한 제대로 된 사죄도 없었다"며 "당내 2차 가해를 적극적으로 막는 조처를 하지 않았다. 이 문제를 회피하고 외면할 수 있지 않을까 하는 오만함이었다"고 비판했다. 아울러 "추미애-윤석열 갈등으로 점철된 (검찰개혁) 추진과정에서 국민들의 공감대를 잃었고 오만과 독선으로 보일 수 있는 행동들이 국민들께 피로와 염증을 느끼게 했음에도 그것이 개혁적 태도라 오판했다"고 말했다. 또 "조국 전 장관이 검찰개혁의 대명사라고 생각했고 검찰의 부당한 압박에 밀리면 안 된다고 판단했지만 그 과정에서 수많은 국민들이 분노하고 분열했다"며 "오히려 검찰개혁의 당위성과 동력을 잃은 것은 아닌가 뒤돌아보고 반성한다"고 말했다. 이들은 "내로남불의 비판을 촉발시킨 여당 인사들의 재산 증식과 이중적 태도에도 국민에게 들이대는 냉정한 잣대와 조치를 들이대지 못하고 억울해하며 변명으로 일관해왔음을 인정한다"며 "분노하셨을 국민께 사과드린다"고 말했다. 의원들은 "민주화를 이뤄낸 국민의 위대함은 민주당만

In [ ]:
rtn = summarize_test(test1)

For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://github.com/hyunwoongko/python-mecab-kor
- konlpy.tag.Mecab: https://konlpy.org/en/latest/api/konlpy.tag/#mecab-class



[ 1 ] "가르치려 한 오만함, 청년과 단절 원인" 반성 "박원순 피해자 제대로 된 사과와 반성 이뤄지지 않았어" 이소영 의원을 비롯한 더불어민주당 2030의원들이 9일 오전 서울 여의도 국회 소통관에서 '더불어민주당 2030의원 입장문' 발표를 기자회견을 하고 있다.
[ 2 ] 민주당 오영환·이소영·장경태·장철민·전용기 의원은 이날 오전 국회 소통관에서 기자회견을 열고 "선거 유세 현장과 삶의 현장에서 만난 20대 30대 청년들은 민주당에 싸늘하고 무관심했고 지난 1년 동안 많은 분의 마음이 돌아섰음을 현장에서 느꼈다"고 말했다.
[ 3 ] 이들은 "이번 재보궐선거를 치르게 된 원인이 우리 당 공직자의 성 비위 문제였음에도 불구하고 우리 당은 당헌·당규를 개정해 후보를 내고 피해자에 대한 제대로 된 사죄도 없었다"며 "당내 2차 가해를 적극적으로 막는 조처를 하지 않았다. 이 문제를 회피하고 외면할 수 있지 않을까 하는 오만함이었다"고 비판했다.


In [ ]:
df['Summary'] = df['Text'].apply(lambda x: summarize_test(x))
df

[ 1 ] 민주당 오영환·이소영·장경태·장철민·전용기 의원은 이날 오전 국회 소통관에서 기자회견을 열고 "선거 유세 현장과 삶의 현장에서 만난 20대 30대 청년들은 민주당에 싸늘하고 무관심했고 지난 1년 동안 많은 분의 마음이 돌아섰음을 현장에서 느꼈다"고 말했다.
[ 2 ] 이들은 "이번 재보궐선거를 치르게 된 원인이 우리 당 공직자의 성 비위 문제였음에도 불구하고 우리 당은 당헌·당규를 개정해 후보를 내고 피해자에 대한 제대로 된 사죄도 없었다"며 "당내 2차 가해를 적극적으로 막는 조처를 하지 않았다. 이 문제를 회피하고 외면할 수 있지 않을까 하는 오만함이었다"고 비판했다.
[ 3 ] "가르치려 한 오만함, 청년과 단절 원인" 반성 "박원순 피해자 제대로 된 사과와 반성 이뤄지지 않았어" 이소영 의원을 비롯한 더불어민주당 2030의원들이 9일 오전 서울 여의도 국회 소통관에서 '더불어민주당 2030의원 입장문' 발표를 기자회견을 하고 있다.
[ 1 ] 동영상 뉴스                  '조국 사태'로 공정 가치 문제 본격적으로 불거져 인국공 사태·윤미향·박원순까지 공정 문제 지속 LH 사태로 文 정부 표방 공정 가치 또 흔들려 민주당 의원들, '조국 지키기' 첫 공개 비판 [앵커] 더불어민주당 2030 세대 의원들이 총선 참패의 원인은 민주당 내부에 있다며 반성문을 내놨습니다.
[ 2 ] 가장 눈에 띄는 건 민주당 안에서는 금기처럼 치부되던 조국 사태에 대한 첫 반성입니다.
[ 3 ] 결국, 그동안 민주당 지지 성향으로 분류되던 2, 30대가 등을 돌리며 선거 패배의 주요 요인으로 꼽혔습니다.
[ 1 ] 연합뉴스     더불어민주당 초선이자 20∼30대 의원인 오영환·이소영·장경태·장철민·전용기 의원 5명은 9일 입장문을 내 “민주당 참패 원인은 저희들을 포함한 민주당의 착각과 판에 있었음을 자인한다”고 밝혔다.
[ 2 ] 이들은 국회 소통관에서 입장문을 발표하고 “이번 재보궐 선거를 치르게 된 원인이 우리 당 공직자의 성 비위 문제였음에도 

,Date,Publisher,Comments,Title,Text,Topic,Summary
0,2021-04-09,뉴스1,1868,"與 2030 의원들 ""조국 관련, 사과할 용의 있어…착각·오판 자인""(종합)","""가르치려 한 오만함, 청년과 단절 원인"" 반성 ""박원순 피해자 제대로 된 사과와 ...",youth_peak1,[민주당 오영환·이소영·장경태·장철민·전용기 의원은 이날 오전 국회 소통관에서 기자...
1,2021-04-09,YTN,684,민주당 2030 의원들의 반성문...'조국 사태' 첫 공개 반성,동영상 뉴스 '조국 사태'로 공정 가치 문제 본격적으...,youth_peak1,[동영상 뉴스 '조국 사태'로 공정 가치 문제 본격적...
2,2021-04-09,한겨레,392,민주당 2030 의원들 “야당·언론·청년 때문에 참패? 동의 못해”,"오영환·이소영·장경태·장철민·전용기 등 입장문 발표 더불어민주당 전용기, 오영환, ...",youth_peak1,[연합뉴스 더불어민주당 초선이자 20∼30대 의원인 오영환·이소영·장경태·장...
3,2022-03-09,JTBC,963,"윤석열 ""투표율, 기대 미치지 못해""…여러차례 독려 메시지",동영상 뉴스 [앵커] 이번엔 국민의힘 상황실로 가보겠...,youth_peak2,[동영상 뉴스 [앵커] 이번엔 국민의힘 상황실로 가보...
4,2022-03-10,KBS,824,젊은 층 표심은? ‘이대남’은 윤석열·‘이대녀’는 이재명,[앵커] 이번 선거 또 하나 관심은 MZ 세대라고 불리는 젊은 층 표심이 어디로 ...,youth_peak2,[방송 3사 출구조사를 통해 분석해보니 2030 젊은 층 유권자는 남녀 성별에 따라...
5,2022-03-09,동아일보,681,"“첫 투표, 진정한 국민 된 기분… 집값-일자리 해결할 대통령 원해”",만 18세 유권자 20명 ‘이런 대통령을 원한다’ 경기 군포 수리고 3학년 김윤산 ...,youth_peak2,[만 18세 유권자 20명 ‘이런 대통령을 원한다’ 경기 군포 수리고 3학년 김윤산...
6,2023-01-12,한겨레,241,‘빌라왕’ 문자 받은 배우…햄버거 알바하던 손이 덜덜 떨려도,[한겨레21] 내 곁에 산재 반지하 전세보증금 7천만원 피해 34살 연극배우의 꿈 ...,rent_peak1,"[버거킹을 거쳐 맥도날드까지 6년을 일했다., [한겨레21] 내 곁에 산재 반지하 ..."
7,2023-01-11,머니투데이,155,"빌라왕 전세사기 피해…""보증 미가입자도 구제""","[국토부 ""1%대 긴급 저리대출·LH 임시거처 등 제공"" 임차인 ""전세금 전액 받을...",rent_peak1,"[[국토부 ""1%대 긴급 저리대출·LH 임시거처 등 제공"" 임차인 ""전세금 전액 받..."
8,2023-01-12,MBC,146,'479채 빌라왕' 전재산 1천만 원‥10명 추적,[뉴스투데이] ◀ 앵커 ▶ 잇따라 터져나오는 빌라왕들의 전세사기. 같은 수법의 사기...,rent_peak1,[MBC는 2015년부터 2018년 사이 빌라를 집중적으로 매입한 뒤 보증 사고를 ...
9,2023-04-18,동아일보,1061,“난 의지할 부모도 없다” 유서…쓰레기봉투엔 정신과 약봉지,두달새 3명째 청년들 앗아간 전세사기 인천 31세 피해여성 숨진채 발견 “다신 이런...,rent_peak2,[두달새 3명째 청년들 앗아간 전세사기 인천 31세 피해여성 숨진채 발견 “다신 이...


In [ ]:
df['Summary'][2]

['연합뉴스     더불어민주당 초선이자 20∼30대 의원인 오영환·이소영·장경태·장철민·전용기 의원 5명은 9일 입장문을 내 “민주당 참패 원인은 저희들을 포함한 민주당의 착각과 판에 있었음을 자인한다”고 밝혔다.',
 '이들은 국회 소통관에서 입장문을 발표하고 “이번 재보궐 선거를 치르게 된 원인이 우리 당 공직자의 성 비위 문제였음에도 불구하고 당은 당헌, 당규를 개정해 후보를 내고 피해자에 대한 제대로 된 사죄도 없었으며, 당내 2차 가해를 적극적으로 막는 조치를 취하지 않았다”며 “이 문제를 회피하고 외면할 수 있지 않을까 하는 오만함이었다”고 인정했다.',
 '그동안 민주당에서 초선 의원들이 당내 현안에 비판적인 입장을 밝힌 것은 이번이 처음으로, 패배 뒤 뒤늦은 반성이라는 지적도 나온다.']

In [ ]:
# # # export results
# df.to_excel('../2023_News/Datasets/summ/news_subset_summ.xlsx', index=False)